In [12]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [13]:
NDE = 10
NLJ = 10
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [14]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [15]:
# Parameters
ffpkl = "mdcm_pbe0dz_FOE.ff.pkl"


#  LJ fit


In [16]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ELEC harmonic first_order_energy (jax_coloumb: True)

In [17]:
# _c.targets

In [18]:
fit_repeat(_c, 
           NLJ, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [19]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
2,15.544575,562,981,0,True,Optimization terminated successfully.,"[0.7263866797246101, 2.499930517663357, 0.0010...","([[0.7263866797246101, 2.499930517663357, 0.00..."
6,15.734114,812,1370,0,True,Optimization terminated successfully.,"[0.7521331606857351, 2.4041202948642186, 0.001...","([[0.7521331606857351, 2.4041202948642186, 0.0..."
5,16.677267,777,1321,0,True,Optimization terminated successfully.,"[0.13674618828126137, 2.4999999672660786, 0.10...","([[0.13674618828126137, 2.4999999672660786, 0...."
9,18.654282,566,984,0,True,Optimization terminated successfully.,"[0.06404095940939791, 2.3646748192715457, 0.45...","([[0.06404095940939791, 2.3646748192715457, 0...."
4,166.175232,406,712,0,True,Optimization terminated successfully.,"[1.809429717309692, 1.1063020563407582, 0.0010...","([[1.809429717309692, 1.1063020563407582, 0.00..."
3,166.764923,330,610,0,True,Optimization terminated successfully.,"[1.8101324659042635, 1.1028255535722296, 0.001...","([[1.8101324659042635, 1.1028255535722296, 0.0..."
0,7296.356445,138,271,0,True,Optimization terminated successfully.,"[1.3352430494656335, 0.05, 0.5, 0.001]","([[1.3352430494656335, 0.05, 0.5, 0.001], [1.3..."
1,8578.110352,40,132,0,True,Optimization terminated successfully.,"[0.050000000000000044, 0.07445537583263218, 0....","([[0.050000000000000044, 0.07445537583263218, ..."
7,8578.110352,41,134,0,True,Optimization terminated successfully.,"[0.050000000000000044, 0.05, 0.229271481923449...","([[0.050000000000000044, 0.05, 0.2292714819234..."
8,8578.110352,44,141,0,True,Optimization terminated successfully.,"[0.050000000000000044, 0.05, 0.5, 0.5]","([[0.050000000000000044, 0.05, 0.5, 0.5], [0.0..."


#  DE fit


In [20]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [21]:
fit_repeat(_c, 
           NDE, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [22]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
3,7.019758,933,1521,0,True,Optimization terminated successfully.,"[1.485504052792685, 2.1190064313887707, 0.0026...","([[1.485504052792685, 2.1190064313887707, 0.00..."
4,7.510250,1149,1877,0,True,Optimization terminated successfully.,"[0.37587472935477995, 2.4892262484327228, 0.43...","([[0.37587472935477995, 2.4892262484327228, 0...."
6,7.690612,611,1030,0,True,Optimization terminated successfully.,"[0.5497994493091545, 2.281821827815243, 0.2219...","([[0.5497994493091545, 2.281821827815243, 0.22..."
1,11.668004,753,1241,0,True,Optimization terminated successfully.,"[0.5274496096968279, 2.283195415985593, 0.5, 0...","([[0.5274496096968279, 2.283195415985593, 0.5,..."
2,12.164757,1012,1632,0,True,Optimization terminated successfully.,"[2.230743990208236, 1.666171080244243, 0.00104...","([[2.230743990208236, 1.666171080244243, 0.001..."
8,13.239778,1036,1708,0,True,Optimization terminated successfully.,"[0.050637234069478065, 2.5, 0.1417503584432692...","([[0.050637234069478065, 2.5, 0.14175035844326..."
5,15.347434,1175,1883,0,True,Optimization terminated successfully.,"[1.6869890553676474, 1.7960610758257083, 0.002...","([[1.6869890553676474, 1.7960610758257083, 0.0..."
7,22.040796,557,941,0,True,Optimization terminated successfully.,"[2.026288651861771, 1.7830168725059539, 0.0066...","([[2.026288651861771, 1.7830168725059539, 0.00..."
0,22.220839,536,945,0,True,Optimization terminated successfully.,"[0.5065461204054236, 1.9670550962785294, 0.113...","([[0.5065461204054236, 1.9670550962785294, 0.1..."
9,8578.110352,55,226,0,True,Optimization terminated successfully.,"[0.2118280482928424, 0.06502377581593105, 0.35...","([[0.2118280482928424, 0.06502377581593105, 0...."
